# Assessment of vancomycin vs. another antibiotic

In clinical practice, the pertinent question is often not "should I use vancomycin?", but rather "should I use vancomycin or this other antibiotic?". This notebook aims to quantify the risk of nephrotoxicity when using vancomycin over the other antibiotic.


## Definitions

* **drug on admission:** patient received medication order -12 to 12 hours upon admission to the ICU
* **baseline creatinine:** first creatinine value between -12 to 12 hours upon admission to the ICU
* **AKI:** following KDIGO guidelines using only creatinine, any instance of AKI between 2-7 days after their ICU admission.

KDIGO guidelines for AKI are: >= 50% change from baseline over 7 days, or absolute increase of 0.3 in creatinine over 48 hours.

## 0. Setup

In [ ]:
# Must install pandas-gbq. Link: https://pandas-gbq.readthedocs.io/en/latest/install.html#pip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# helper functions stored in local py file
import utils

project_id='lcp-internal'

# Helper function to read data from BigQuery into pandas dataframes.
def run_query(query):
    return pd.io.gbq.read_gbq(query,
                              project_id=project_id, verbose=False,
                              dialect='standard')

# 1. Extract data and apply exclusions

For more detail on exclusions, see the main vancomycin analysis notebook.

In [ ]:
# cohort with exclusions applied
query = """
SELECT co.*
FROM `lcp-internal.vanco.cohort` co
"""
co = run_query(query)

# covariates from APACHE table
query = """
SELECT dem.*
FROM `hst-953-2018.team_i.demographics` dem
"""
dem = run_query(query)

# abx drug doses
query = "SELECT * FROM `lcp-internal.vanco.vanco`"
va = run_query(query)

query = "SELECT * FROM `lcp-internal.vanco.cefepime`"
ce = run_query(query)

query = "SELECT * FROM `lcp-internal.vanco.zosyn`"
zo = run_query(query)

# AKI
query = """
SELECT 
  patientunitstayid
  , chartoffset
  , creatinine, creatinine_reference, creatinine_baseline
  , aki_48h, aki_7d
FROM `lcp-internal.vanco.aki`
"""
aki = run_query(query)

## Merge data

The antibiotic administration data extracted above has all administrations from time 0.
Thus, three dataframes need to be extracted before we can merge data:

1. Extract vanco_adm and vanco_wk
2. Extract cefepime_adm and cefepime_wk
3. Extract zosyn_adm and zosyn_wk

In [ ]:
v_df = utils.extract_adm_and_wk(va, 'vanco')
c_df = utils.extract_adm_and_wk(ce, 'cefepime')
z_df = utils.extract_adm_and_wk(zo, 'zosyn')

v_df.head()

## Apply exclusions and create final dataframe

In [ ]:
# drop exclusions
idxKeep = co['patientunitstayid'].notnull()
for c in co.columns:
    if c.startswith('exclude_'):
        idxKeep = idxKeep & (co[c]==0)

# combine data into single dataframe
df = co.loc[idxKeep, ['patientunitstayid']].merge(dem, how='inner', on='patientunitstayid')

# add abx administration
df = df.merge(v_df, how='left', on='patientunitstayid')
df = df.merge(c_df, how='left', on='patientunitstayid')
df = df.merge(z_df, how='left', on='patientunitstayid')

# if the patient unit stay ID was not present for above join, they did not receive the abx
abx_columns = list(v_df.columns) + list(c_df.columns) + list(z_df.columns)
for c in abx_columns:
    df[c].fillna(0, inplace=True)
    df[c] = df[c].astype(int)

aki_grp = aki.groupby('patientunitstayid')[['creatinine', 'aki_48h', 'aki_7d']].max()
aki_grp.reset_index(inplace=True)
df = df.merge(aki_grp, how='inner', on='patientunitstayid')

df['aki'] = ((df['aki_48h'] == 1) | (df['aki_7d'] == 1)).astype(int)

print('{} patients.'.format(df.shape[0]))

print('Antibiotic use on admission to ICU:')
for abx in ['vanco', 'cefepime', 'zosyn']:
    N = df[abx + '_adm'].sum()
    mu = N/df.shape[0]*100.0
    print(f'  {N:5d} ({mu:4.1f}%) - {abx}')
    
    
print('\nConcurrent antibiotic use:')
for abx1, abx2 in [['vanco', 'cefepime'], ['vanco', 'zosyn']]:
    N = ((df[abx1 + '_adm'] == 1) & (df[abx2 + '_adm'] == 1)).sum()
    mu = N/df.shape[0]*100.0
    print(f'  {N:5d} ({mu:4.1f}%) - {abx1} & {abx2}')
    
    
df.head()

# Propensity matching

Define the dataframes used for (1) vanco only, (2) vanco + zosyn, and (3) vanco + cefepime.

TODO: should the control group always exclude *all* other ABX, or just the one in question?

### Vanco + Zosyn vs. Vanco + Cefepime

In [ ]:
vanco_zosyn = df[(df['vanco_adm'] == 1) & (df['zosyn_adm'] == 1) & (df['cefepime_adm'] == 0)]
vanco_cefepime = df[(df['vanco_adm'] == 1) & (df['zosyn_adm'] == 0) & (df['cefepime_adm'] == 1)]
utils.match_and_print_or(exposure=vanco_zosyn, control=vanco_cefepime)

### Vanco vs. Vanco + Zosyn Analysis

In [ ]:
vanco_only = df[(df['vanco_adm'] == 1) & (df['zosyn_adm'] == 0)]
vanco_zosyn = df[(df['vanco_adm'] == 1) & (df['zosyn_adm'] == 1)]
utils.match_and_print_or(exposure=vanco_zosyn, control=vanco_only)

### Vanco vs. Vanco + Cefepime Analysis

In [ ]:
# Vanco + Cefepime Analysis
vanco_only = df[(df['vanco_adm'] == 1) & (df['cefepime_adm'] == 0)]
vanco_cefepime = df[(df['vanco_adm'] == 1) & (df['cefepime_adm'] == 1)]
utils.match_and_print_or(exposure=vanco_cefepime, control=vanco_only)